In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(3, 6, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2)
        
        self.conv_trans1 = nn.ConvTranspose2d(6, 3, 4, 2, 1)
        self.conv_trans2 = nn.ConvTranspose2d(3, 1, 4, 2, 1)
        
    def forward(self, x):
        x = F.relu(self.pool1(self.conv1(x)))
        x = F.relu(self.pool2(self.conv2(x)))        
        x = F.relu(self.conv_trans1(x))
        x = self.conv_trans2(x)
        return x



In [ ]:
import torchvision

n_epochs = 3
batch_size_train = 8
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

In [ ]:
model = MyModel()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 1
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data)
        loss.backward()
        optimizer.step()
        
    print('Epoch {}, Batch idx {}, loss {}'.format(
        epoch, batch_idx, loss.item()))



In [ ]:
print(model)

In [ ]:
def normalize_output(img):
    img = img - img.min()
    img = img / img.max()
    return img

'''
# Plot some images
idx = torch.randint(0, output.size(0), ())
pred = normalize_output(output[idx, 0])
img = data[idx, 0]

fig, axarr = plt.subplots(1, 2)
axarr[0].imshow(img.detach().numpy())
axarr[1].imshow(pred.detach().numpy())
'''
# Visualize feature maps
activation = {}
def get_activation(name):
    def hook(model,input, output):
        activation[name] = output.detach()
    return hook

model.conv1.register_forward_hook(get_activation('conv1'))
model.conv2.register_forward_hook(get_activation('conv2'))
model.conv_trans1.register_forward_hook(get_activation('conv_trans1'))
model.pool1.register_forward_hook(get_activation('pool1'))

In [ ]:
dataset = torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

'''
data, _ = dataset[1]
data.unsqueeze_(0)
output = model(data)

act = activation['conv1'].squeeze()
fig, axarr = plt.subplots(act.size(0))
for idx in range(act.size(0)):
    axarr[idx].imshow(act[idx])
'''

In [ ]:
import numpy as np
m = nn.ReLU()
def convert_to_ones(inp):
    #relu_inp = m(inp)
    inp[inp <= 100] = 0
    inp[inp > 100] = 1
    return inp

In [ ]:
def diff(name, indx1, indx2):
    def convert(index, name):
        data, _ = dataset[index]
        data.unsqueeze_(0)
        output = model(data)
        act = activation[name].squeeze()
        fig, axarr = plt.subplots(1)
        tensor = act[0]
        for idx in range(1, act.size(0)):
            tensor += act[idx]
        #tensor = convert_to_ones(tensor)
        plt.imshow(tensor)
        return tensor
    
    tensor1 = convert(indx1, name)
    tensor2 = convert(indx2, name)
    intersection = torch.sum(tensor1 == tensor2).item() / (tensor1.shape[0] * tensor1.shape[1])
    return intersection
print(diff('conv1', 1, 4))
print(diff('conv2', 1, 4))
print(diff('conv_trans1', 1, 4))
print(diff('pool1', 1, 4))

# Using a classifier network

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #print(x.shape)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
            (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), 'results/model.pth')
            torch.save(optimizer.state_dict(), 'results/optimizer.pth')
    

def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            test_loss /= len(test_loader.dataset)
            test_losses.append(test_loss)
            print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))



In [ ]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

In [ ]:
def convert_to_classid(inp, class_id):
    inp[inp <= 100] = 0
    inp[inp > 100] = class_id * 10
    return inp

def diff2(name, indx1, indx2):
    def convert(index, name):
        data, _ = dataset[index]
        data.unsqueeze_(0)
        output = network(data)
        act = activation[name].squeeze()
        fig, axarr = plt.subplots(1)
        tensor = act[0]
        #for idx in range(1, act.size(0)):
        #    tensor += act[idx]
        #tensor = convert_to_ones(tensor)
        plt.imshow(tensor)
        return tensor
    
    tensor1 = convert(indx1, name)
    tensor2 = convert(indx2, name)
    intersection = torch.sum(tensor1 == tensor2).item() / (tensor1.shape[0] * tensor1.shape[1])
    return intersection

In [ ]:
def normalize_output(img):
    img = img - img.min()
    img = img / img.max()
    return img

# Visualize feature maps
activation = {}
def get_activation(name):
    def hook(network,input, output):
        activation[name] = output.detach()
    return hook

network.conv1.register_forward_hook(get_activation('conv1'))
network.conv2.register_forward_hook(get_activation('conv2'))
network.conv2_drop.register_forward_hook(get_activation('conv2_drop'))
network.fc1.register_forward_hook(get_activation('fc1'))
network.fc2.register_forward_hook(get_activation('fc2'))

In [ ]:
ind1 = 1
ind2 = 21
print(diff2('conv1', ind1, ind2))
print(diff2('conv2', ind1, ind2))
print(diff2('conv2_drop', ind1, ind2))
#print(diff2('fc1', 1, 4))

In [ ]:
data, _ = dataset[18]
data.unsqueeze_(0)
output = network(data)
act = activation['conv1'].squeeze()
fig, axarr = plt.subplots(1)
test = act[0]
for idx in range(1, act.size(0)):
    test += act[idx]
#test = convert_to_classid(test,output.argmax().item())
plt.imshow(test)

In [ ]:
output.argmax().item()

In [ ]:
activation.keys()

In [ ]:
act = activation['conv2'].squeeze()
fig, axarr = plt.subplots(act.size(0))
for idx in range(act.size(0)):
    axarr[idx].imshow(act[idx])

In [ ]:
model.conv_trans1.register_forward_hook(get_activation('conv_trans1'))

In [ ]:
act = activation['conv_trans1'].squeeze()
fig, axarr = plt.subplots(act.size(0))
for idx in range(act.size(0)):
    axarr[idx].imshow(act[idx])

In [ ]:
act = activation['pool1'].squeeze()
fig, axarr = plt.subplots(act.size(0))
for idx in range(act.size(0)):
    axarr[idx].imshow(act[idx])

In [ ]:
activation.keys()